<a href="https://colab.research.google.com/github/reiniscimurs/gnn_with_pytorch/blob/main/section_3/01_simple_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple model training with Deep Learning
Training a neural network for handwritten character recognition.
Since we are using GPU for learning, select 'GPU' in 'Hardware Accelerator' under 'Edit' -> 'Notebook Settings'.

## Loading Data
Use `torchvision.datasets` to load handwritten character data and configure a DataLoader.
By using DataLoader, tasks such as data loading and implementation of mini-batch methods become significantly easier.
In addition to MNIST, `torchvision.datasets` provides various other datasets.
https://pytorch.org/docs/stable/torchvision/datasets.html


In [ ]:
import torch
from torchvision.datasets import MNIST  # Handwritten character image data
from torchvision import transforms
from torch.utils.data import DataLoader

# Obtain training data
mnist_train = MNIST("./data",
                    train=True, download=True,
                    transform=transforms.ToTensor())
# Obtain test data
mnist_test = MNIST("./data",
                   train=False, download=True,
                   transform=transforms.ToTensor())
print("Number of training data:", len(mnist_train), "Number of test data:", len(mnist_test))

# Configure DataLoader
img_size = 28
batch_size = 256
train_loader = DataLoader(mnist_train,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(mnist_test,
                         batch_size=batch_size,
                         shuffle=False)

The size of the handwritten character images is 28×28.



## Model Construction
We will construct the model as a class inheriting from the nn.Module module.
By using .cuda(), the model calculations will be performed on the GPU.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(img_size*img_size, 1024)  # Fully connected layer
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, img_size*img_size)  # Batch size × number of inputs
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.cuda()  # Enable GPU
print(net)

## Training
Using DataLoader, extract mini-batches (chunks of data) for training and evaluation.

Since training is performed multiple times with mini-batches within an epoch, it implies the implementation of mini-batch training.


In [ ]:
from torch import optim

# Cross-entropy loss function
loss_fnc = nn.CrossEntropyLoss()

# SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Loss logs
record_loss_train = []
record_loss_test = []

# Training
for i in range(10):  # 10 epochs of training
    net.train()  # Training mode
    loss_train = 0
    for j, (x, t) in enumerate(train_loader):  # Extract mini-batch (x, t)
        x, t = x.cuda(), t.cuda()  # GPU support
        optimizer.zero_grad()  # ① Initialize gradients
        y = net(x)  # ② Obtain predictions through forward propagation
        loss = loss_fnc(y, t)  # ③ Calculate loss from predictions and correct labels
        loss_train += loss.item()
        loss.backward()  # ④ Compute gradients through backpropagation from the loss
        optimizer.step()  # ⑤ Update parameters through the optimization algorithm
    loss_train /= j+1
    record_loss_train.append(loss_train)

    net.eval()  # Evaluation mode
    loss_test = 0
    for j, (x, t) in enumerate(test_loader):  # Extract mini-batch (x, t)
        x, t = x.cuda(), t.cuda()
        y = net(x)
        loss = loss_fnc(y, t)
        loss_test += loss.item()
    loss_test /= j+1
    record_loss_test.append(loss_test)

    if i % 1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)

## Missmatch visualization
Plotting the progression of errors for training and test data.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## Accuracy

To assess the performance of the model, we will measure the accuracy using the test data.

In [ ]:
correct = 0
total = 0
for i, (x, t) in enumerate(test_loader):
    x, t = x.cuda(), t.cuda()  # GPU support
    x = x.view(-1, img_size*img_size)
    y = net(x)
    correct += (y.argmax(1) == t).sum().item()
    total += len(x)
print("Accuracy:", str(correct/total*100) + "%")